ChEn-3170: Computational Methods in Chemical Engineering Fall 2018 UMass Lowell; Prof. V. F. de Almeida **24Sep2018**

# 05. Computational Linear Algebra Fundamentals

---
## Table of Contents
* [Theory](#theory)
* [Matrix multiplication](#matmul)
---

## Theory<a id="theory"></a>
The notes found [here](https://studentuml-my.sharepoint.com/:o:/g/personal/valmor_dealmeida_uml_edu/ErfDAD_jL1tHkDrx29w89-MBcOgK4JAnEYSheRoxkL_0sw?e=fpzG5K) cover basic elements of lineary system of algebraic equations. Their representation in row and column formats and the insight gained from them when searching for a solution of the system.